# <center>NLP: Topic Modeling for Riyadh Newspaper Articles</center>



**DATA**: We are planning to use our previously **scraped** data from the Riyadh Newspaper website **in Arabic**, ~3GB, plus some additional data (MetaData) that could add a meaningful sense to our project, see [Riyadh Newspaper website](http://www.alriyadh.com/1814297) to understand how we scraped the data.

**EXPECTED OUTPUT**: At the end of this project we expect to be done NLP **Topic Modeling** on Riyadh Newspaper articles, and all it's required preproccesing steps in addition to approperiate visualization.

**BONUS TASKS**: 
- Scraping ~3GB Data. 
- Using SQL Database for data storage. 
- As a challenge, we used data written in Arabic.

In [1]:
import pandas  as pd
import numpy as np
import pyodbc
import nltk

### reading from Database

In [2]:
# server = 't5.database.windows.net'
# database = 'T5'
# username = 't5'
# password = 'My404Data'

# # dfs2.columns=dfs2.columns.str.strip()

# cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()

In [3]:
# df= pd.read_sql("select * from articles",cnxn)
# df
# df.to_csv("articles.csv", encoding='utf-8-sig')

In [4]:
# df2= pd.read_sql("select top 10000 * from texts",cnxn)
# df2
# df2.to_csv("texts.csv",encoding='utf-8-sig')

### reading CSVs files

In [5]:
df_articles = pd.read_csv('articles.csv')

In [6]:
df_texts = pd.read_csv('texts.csv')

### preveiw

In [7]:
df_articles.head(5)

,Unnamed: 0,A_ID,A_Link,A_LinkNum,A_Timestr,A_Cat,A_Auth,A_Title1,A_Title2,A_Paragraphs,A_Words,A_Characters
0,0,1,http://www.alriyadh.com/1814297,1814297,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,"الرياض - ""الرياض""",إلى ما بعد اجتماع مجموعة أوبك بلس,السعودية تؤجل الإعلان عن أسعار الخام لشهر مايو,8,339,1962
1,1,2,http://www.alriyadh.com/1814297,1814297,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,"الرياض - ""الرياض""",إلى ما بعد اجتماع مجموعة أوبك بلس,السعودية تؤجل الإعلان عن أسعار الخام لشهر مايو,8,339,1962
2,2,3,http://www.alriyadh.com/1814296,1814296,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,وادي الدواسر - سعود آل مسيب,NaN,تعليم وادي الدواسر يبدأ المرحلة الثانية من خطة...,3,183,1162
3,3,4,http://www.alriyadh.com/1814296,1814296,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,وادي الدواسر - سعود آل مسيب,NaN,تعليم وادي الدواسر يبدأ المرحلة الثانية من خطة...,3,183,1162
4,4,5,http://www.alriyadh.com/1814295,1814295,&nbsp; \n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\...,المحليات,الرياض - محمد الحيدر,من برنامج إعانة الباحثين عن عمل,"""هدف"" يودع 446 مليون ريال في حسابات المستفيدين",4,190,1148


In [8]:
df_texts.head(5)

,Unnamed: 0,T_ID,T_AID,T_Text
0,0,87160,174309,\nعلى الرغم من التحذيرات الشديدة والمتكررة الت...
1,1,87164,174317,\nأدى محافظ ضمد عبدالله خالد البراق عقب صلاة ع...
2,2,87165,174319,\nرفع رئيس مجلس إدارة الهيئة العامة للولاية عل...
3,3,87168,174325,\nوجه صاحب السمو الملكي الأمير د. فيصل بن مشعل...
4,4,87169,174327,\nاستقبل صاحب السمو الملكي الأمير فيصل بن خالد...


# Article Example

In [9]:
df_texts['T_Text'][0]

'\nعلى الرغم من التحذيرات الشديدة والمتكررة التي يوجهها الدفاع المدني للمواطنين قبل وبعد هطول الأمطار لتوخي الحذر والابتعاد عن أماكن تجمعات السيول وبطون الأودية ورغم ما تتناقله وسائل التواصل الاجتماعي من مقاطع مؤلمة ومفجعة لأشخاص جرفتهم السيول بمركباتهم ولقوا حتفهم بأبشع الصور وأسوأء المناظر، إلاّ أنه مازال هناك الكثير من المراهقين ومن فاقدي البصيرة المتهورين تستهويهم المغامرات المهلكة داخل بطون الأودية والأماكن التي غمرتها مياه الأمطار، غير مبالين بخطورة تلك الأفعال وشناعة تلك التصرفات العبثية، التي ربما يفقدون حياتهم بسببها في أدنى لحظة.\nوفي كل موسم من مواسم الأمطار تتجدد ظاهرة التهور والمغامرات في السيول وتتجدد معها معاناة رجال الدفاع المدني الذين مهما كان لديهم من إمكانات لن يستطيعوا تغطية جزء مساحته مئات الكيلومترات من براري المملكة الشاسعة التي تتنوع فيها التضاريس المعقدة.\nماذا بعد\nوتداولت شبكات التواصل الاجتماعي مقاطع فيديوهات لبعض المتهورين وهم يقتحمون الأودية مع سبق الإصرار، غير مبالين بتحذيرات الدفاع المدني وغير مبالين بالأشخاص الموجودين بتلك المواقع الذين حاولوا ثنيهم عن 